# Improved Integration of GRASS and Jupyter

As part of Google Summer of Code 2021, we've been working to shorten and simplify the launch of GRASS in Jupyter and imporve the map displays. It is experimentally included in version 8.0 as a preview and will be officially available in version 8.2. You can find out more about the project and follow the progress on the [GRASS wiki page](https://trac.osgeo.org/grass/wiki/GSoC/2021/JupyterAndGRASS).

In addition to simplifying the launch of *GRASS GIS* with `init()`, `grass.jupyter` has two main dislay classes, `GrassRenderer` and `InteractiveMap`. Using the *GRASS* rendering engine in the background, `GrassRenderer` creates maps as PNG images. `InteractiveMap` displays *GRASS GIS* rasters and vectors with [*folium*](http://python-visualization.github.io/folium/), a [*leaflet*](https://leafletjs.com/) library for *Python*.

This interactive notebook is available online thanks to the [https://mybinder.org](Binder) service. To run the select part (called a *cell*), hit `Shift + Enter`.

## Start GRASS GIS

In [ ]:
import os
import subprocess
import sys

# Add GRASS to path
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("../../data/grassdata", "nc_basic_spm_grass7", "user1")

# Set computational region to the elevation raster.
gs.run_command("g.region", raster="elevation")

## GRASS Renderer

The `GrassRenderer` class creates and displays GRASS maps as PNG images. There are two ways to add elements to the display. First, the name of the *GRASS* display module can be called as an attribute by replacing the "." with "\_" in the module name. For example:
````
m = GrassRenderer()
m.d_rast(map="elevation")
````

Alternatively, *GRASS* display modules can be called with the `run()` method:
````
m = GrassRenderer()
m.run("d.rast", map="elevation")
````

To display the image, call `show()`.

In [ ]:
# Create GrassRenderer instance
img = gj.GrassRenderer()

In [ ]:
# Add a raster, vector and legend to the map
img.d_rast(map="elevation")
img.d_vect(map="streams")
img.d_legend(raster="elevation", at=(55, 95, 80, 84), flags="b")

In [ ]:
# Display map
img.show()

We can also have multiple instances of `GrassRenderer`. Here, we create another map then go back and modify the first map.

In [ ]:
# Make a second instance.
# Just for variety, we'll make this one a different size
img2 = gj.GrassRenderer(height=200, width=220)

In [ ]:
# Add a some layers
# We can also add layers with the run() methods
img2.run("d.rast", map="elevation_shade")
img2.run("d.vect", map="roadsmajor")

# Display second map
img2.show()

In [ ]:
# Then, we return to the first instance and continue to modify and display it
# Notice that layers a drawn in the order they are added
img.run("d.vect", map = "zipcodes", color="red", fill_color="none")
img.show()

## Interactive Map Display

The `InteractiveMap` class displays *GRASS GIS* rasters and vectors with [*folium*](http://python-visualization.github.io/folium/), a [*leaflet*](https://leafletjs.com/) library for *Python*.

In [ ]:
# Create Interactive Map
fig = gj.InteractiveMap(width = 600)

In [ ]:
# Add raster, vector and layer control to map
fig.add_raster("elevation")
fig.add_vector("roadsmajor")
fig.add_layer_control(position = "bottomright")

In [ ]:
# Display map
fig.show()

## Save InteractiveMap as HTML

To share or embed the map in a website, we can export it has an HTML file.

In [ ]:
fig.save(filename="test_map.html")

## GRASS 3D Renderer

The `Grass3dRenderer` class creates 3D visualizations as PNG images. The *m.nviz.image* module is used in the background and the function `render()` accepts parameters of this module.
The `Grass3dRenderer` objects have `overlay` attribute which can be used in the same way as `GrassRenderer` and 2D images on top of the 3D visualization.
To display the image, call `show()`.

First, let's create the object:

In [ ]:
img = gj.Grass3dRenderer()

Now, render a 3D visualization of an elevation raster as a surface colored using, again, the elevation raster:

In [ ]:
img.render(elevation_map="elevation", color_map="elevation", perspective=20)

To add a raster legend on the image as an overlay using the 2D rendering capabilities accessible with `overlay.d_legend`:

In [ ]:
img.overlay.d_legend(raster="elevation", at=(60, 97, 87, 92))

Finally, we show 

In [ ]:
img.show()

Now, let's color the elevation surface using a landuse raster (note that the call to `render` removes the result of the previous `render` as well as the current overlays):

In [ ]:
img.render(elevation_map="elevation", color_map="landuse", perspective=20)
img.show()